<a href="https://colab.research.google.com/github/jsebastianquiroga/Topicos_analitica/blob/main/Notebook/Empirical_network_twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <img style="float: center; padding-right: 10px;" src="https://www.pikpng.com/pngl/b/467-4670553_universidad-javeriana-esslingen-am-neckar-clipart.png" width="100" height="150">



<h1> <strong>Tópicos avanzados en analítica.</strong></h1> 
<h2> Para: Luis Gabriel Moreno Sandoval.</br></h2>
<h2>Integrantes:</br></h2>
<h2>Alejandro Vivas.</br></h2>
<h2>Johan Sebastian Muñoz.</br></h2>
<h2>Juan Sebastián Quiroga Bernal. </h2>

<h1> <strong>Workshop 1: Empirical Network  - Twitter.</strong></h1> 

En el presnte Notebook, se desarrollar una funcion que permita extraer los tuits que se desen con la API de snscrape.


Este docuemnto esta dividio en las siguientes partes:
<ol>
<li> Funcion - Extraer Tuits.. </li>
<li> Creación de Red de los Hastags. </li>
<li> Creación de Red que asocie las palabras mas usadas. </li>
<li> Conclusiones.</li>
</ol>

***

In [ ]:
!pip install snscrape

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#creación de funcion.

#import the libraries to be use.
import snscrape.modules.twitter as sntwitter
import pandas as pd

query = "ptyhon"
tweets = []
limits = 100

for tweet in sntwitter.TwitterSearchScraper(query).get_items():

  if len(tweets) == limits:
    break
  else:
    tweets.append([tweet.date, tweet.user.username, tweet.content])

df =pd.DataFrame(tweets, columns =['Date', 'User', 'Tweet'])

df

  # print(vars(tweet))


<ipython-input-8-dd5c721e3a40>:16: DeprecatedFeatureWarning: content is deprecated, use rawContent instead
  tweets.append([tweet.date, tweet.user.username, tweet.content])


,Date,User,Tweet
0,2023-03-17 05:21:33+00:00,Ptyhon1,@ismailsaymaz Vardır bir bildiği kralın. O gel...
1,2023-03-17 05:20:33+00:00,Ptyhon1,@ReatKanburolu10 @vekilince Bu türkçeyle başka...
2,2023-03-16 11:36:12+00:00,Ptyhon1,@Drr_Nilay Muhalefetin başına gelince halk tep...
3,2023-03-16 11:32:49+00:00,Ptyhon1,@hissiyatlarim Alakaya maydanoz amk
4,2023-03-16 07:37:56+00:00,hidesugar2,project euler590、threadの7番目のpostと同じ解法で解いた。C++で...
...,...,...,...
95,2023-01-25 10:01:55+00:00,BMuhCihanTurunc,Click here to get ptyhon training with the bes...
96,2023-01-25 09:43:05+00:00,StefanCr94,@alex_streza Ptyhon has the easiest syntax.
97,2023-01-24 11:59:39+00:00,Mahmoud24804495,@MrXroboT طيب ولو انا عاوز اsolve \nPtyhon اغي...
98,2023-01-23 23:23:33+00:00,vetlanprogramer,初心者がRailsなんて零細なWebベンチャーのRails求人しかない。技術要求は高く、給料...


##2

In [119]:
import snscrape.modules.twitter as sntwitter
import networkx as nx
import plotly.graph_objects as go
import re
import pandas as pd

import snscrape.modules.twitter as sntwitter
import networkx as nx
import plotly.graph_objects as go
import re
import pandas as pd
import plotly.express as px
import datetime as dt
import pytz

class TwitterGraph:
    def __init__(self, query, search_method, max_tweets=100, start_date=None, end_date=None):
        self.query = query
        self.search_method = search_method
        self.max_tweets = max_tweets
        self.start_date = start_date
        self.end_date = end_date
        self.graph = nx.Graph()
        self.data = None

    def _scrape_tweets(self):
        # construct search query based on search method and time range filters
        if self.start_date and self.end_date:
            date_range_query = f'since:{self.start_date} until:{self.end_date}'
            search_query = f'{self.query} {date_range_query}'
        else:
            search_query = self.query

        # create a scraper object based on the search method
        if self.search_method == 'keyword':
            scraper = sntwitter.TwitterSearchScraper(search_query)
        elif self.search_method == 'hashtag':
            scraper = sntwitter.TwitterSearchScraper(f'#{search_query}')
        elif self.search_method == 'username':
            scraper = sntwitter.TwitterUserScraper(search_query)
        elif self.search_method == 'list':
            scraper = sntwitter.TwitterListScraper(search_query)
        elif self.search_method == 'advanced':
            scraper = sntwitter.TwitterSearchScraper(search_query)
        elif self.search_method == 'location':
            scraper = sntwitter.TwitterSearchScraper(search_query)
        else:
            raise ValueError("Invalid search_method. Must be 'keyword', 'hashtag', 'username', 'list', 'advanced', or 'location'.")

        tweets = []
        count = 0
        # fetch tweets until we reach the desired number of tweets
        for tweet in scraper.get_items():
            if count >= self.max_tweets:
                break
            tweets.append(tweet)
            count += 1

        # filter tweets by date range if start_date and end_date are specified
        if self.start_date and self.end_date:
            start_date_dt = dt.datetime.strptime(self.start_date, '%Y-%m-%d').replace(tzinfo=pytz.UTC)
            end_date_dt = dt.datetime.strptime(self.end_date, '%Y-%m-%d').replace(tzinfo=pytz.UTC)
            filtered_tweets = []
            for tweet in tweets:
                if tweet.date >= start_date_dt and tweet.date <= end_date_dt:
                    filtered_tweets.append(tweet)
            tweets = filtered_tweets

        # extract tweet information
        data = []
        for tweet in tweets:
            # extract hashtags
            hashtags = re.findall(r'#\w+', tweet.rawContent)
            for i, h1 in enumerate(hashtags):
                for h2 in hashtags[i+1:]:
                    # create an edge between users who have used the same hashtag
                    self.graph.add_edge(tweet.user.username, h1)
                    self.graph.add_edge(tweet.user.username, h2)
            
            data.append({
                'tweet_id': tweet.id,
                'username': tweet.user.username,
                'content': tweet.rawContent,
                'date': tweet.date,
                'retweets': tweet.retweetCount,
                'likes': tweet.likeCount,
                'quotes': tweet.quoteCount,
                'replies': tweet.replyCount,
                'url': tweet.url
            })

        # Create a DataFrame from the extracted data
        self.data = pd.DataFrame(data)
        return data

    def build_graph(self):
        self._scrape_tweets()
        self.graph.remove_nodes_from(list(nx.isolates(self.graph)))

        # compute node positions for the graph
        node_positions = nx.spring_layout(self.graph)

        # create a dictionary that maps usernames to node positions
        user_positions = {}
        for node, pos in node_positions.items():
            user_positions[node] = pos
            self.graph.nodes[node]['pos'] = pos

        return user_positions
        
    def visualize_graph(self):
        # get the node positions from the built graph
        node_positions = self.build_graph()

        # create a dataframe from the node positions
        pos_df = pd.DataFrame(node_positions, index=['x', 'y']).T
        pos_df.index.name = 'node'

        # add the node positions to the graph
        nx.set_node_attributes(self.graph, pos_df.to_dict('index'))

        # create a plotly figure object
        fig = px.scatter(pos_df, x='x', y='y', text=pos_df.index, custom_data=[pos_df.index])

        # add edges to the figure
        for edge in self.graph.edges:
            x0, y0 = self.graph.nodes[edge[0]]['x'], self.graph.nodes[edge[0]]['y']
            x1, y1 = self.graph.nodes[edge[1]]['x'], self.graph.nodes[edge[1]]['y']
            fig.add_trace(
                go.Scatter(x=[x0, x1], y=[y0, y1], mode='lines', line=dict(color='gray'))
            )

        # configure the figure layout
        fig.update_traces(textposition='top center', marker=dict(size=10, color='lightblue'), hovertemplate='Username: %{customdata[0]}<extra></extra>')
        fig.update_layout(title=f'Twitter Graph for {self.query}', title_font_size=30,
                          xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                          yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                          hovermode='closest')

        # display the figure
        fig.show()

In [120]:
# Instantiate the TwitterGraph class with the desired parameters
tg = TwitterGraph(query='Chelsea', search_method='hashtag', max_tweets=100, start_date='2023-03-15', end_date='2023-03-18')

# Build and visualize the graph
tg.visualize_graph()
